# Domain Adaptation

In [1]:
from dataset import Dataset, build_FIVES_dataloaders, build_hsi_dataloader, HSIDataset
from torch.utils.data import DataLoader
import torch

trainloader_source, validationloader_source, testloader_source = (
    build_FIVES_dataloaders(batch_size=8, width=224, height=224)
)


trainloader_target = build_hsi_dataloader(
    batch_size=8,
    train_split=1,
    val_split=0,
    test_split=0,
    window=(400, 1100),
    exclude_labeled_data=True,
)[0]

path = "./data/helicoid_with_labels"
testset = HSIDataset(path, with_gt=True, window=(400, 1100))
testset.normalize_dataset()
testloader_target = DataLoader(testset, batch_size=1, shuffle=False)
print(len(trainloader_target), len(testloader_target))

Number of samples in the training set: 640, validation set: 60
Number of samples in the test set: 200
4 5


In [ ]:
from segmentation_util import build_segmentation_model, build_criterion, build_optimizer
import matplotlib.pyplot as plt
from torch.autograd import Function
from torchvision import models
import torch.nn as nn
import torch
from domain_adaptation_training import model_pipeline
from HSI_Net import ModelWithDomainAdaptation, DomainClassifierFC

encoder = "resnet34"
architecture = "Unet"
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
lambda_param = 0.5
learning_rate = 0.0157
epochs = 10

segmentation_model = build_segmentation_model(encoder, architecture, device)
domain_classifier = DomainClassifierFC(num_domains=2).to(device)
model = ModelWithDomainAdaptation(
    segmentation_model, lambda_param=lambda_param, domain_classifier=domain_classifier
).to(device)

criterion = build_criterion("Dice")
optimizer = build_optimizer(model, learning_rate=learning_rate, optimizer="Adam")

config = {
    "model": "{architecture}-{encoder}".format(
        architecture=architecture, encoder=encoder
    ),
    "encoder": encoder,
    "architecture": architecture,
    "learning_rate": learning_rate,
    "optimizer": "Adam",
    "lambda_param": lambda_param,
    "batch_size_source": trainloader_source.batch_size,
    "batch_size_target": trainloader_target.batch_size,
    "epochs": epochs,
}

model, train_losses, val_losses_source, val_losses_target = model_pipeline(
    model,
    trainloader_source,
    validationloader_source,
    testloader_source,
    trainloader_target,
    testloader_target,
    criterion,
    optimizer,
    config,
    project="domain-adaptation",
    epochs=epochs,
    model_name=config["model"],
    device=device,
    batch_print=1,
    evaluate=True,
    with_overlays=True,
)

# Plotting the training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label="Combined Training Loss")
plt.plot(val_losses_source, label="Validation Loss Source")
plt.plot(val_losses_target, label="Validation Loss Target")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Curves")
plt.legend()
plt.show()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tim-mach (idp2024). Use `wandb login --relogin` to force relogin


Epoch 1, Train Loss: 0.0000
Evaluating model performance on source data


/home/home/tim_ivan/idp/.venv/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Precision: 0.0055, Recall: 0.4290, F1 Score: 0.0109, Dice Score: 0.0109, Accuracy: 0.5277
Evaluating model performance on target data
Precision: 0.2103, Recall: 0.2039, F1 Score: 0.1962, Dice Score: 0.1962, Accuracy: 0.6845
Epoch 1, Validation Loss Source: 0.8765, Validation Loss Target: 0.6997
Epoch 2, Train Loss: 0.0000
Evaluating model performance on source data
Precision: 0.0055, Recall: 0.4290, F1 Score: 0.0109, Dice Score: 0.0109, Accuracy: 0.5277
Evaluating model performance on target data
Precision: 0.2103, Recall: 0.2039, F1 Score: 0.1962, Dice Score: 0.1962, Accuracy: 0.6845
Epoch 2, Validation Loss Source: 0.8765, Validation Loss Target: 0.6997
Epoch 3, Train Loss: 0.0000
